In [1]:
import os
import numpy as np
import cv2
import math
import SimpleITK as sitk
from skimage.transform import resize
import pandas as pd

# Function to calculate the circumference of an ellipse
def ellipse_circumference(a, b):
    return np.pi * ((3 * (a + b) - np.sqrt((3 * a + b) * (a + 3 * b))) * 0.28)

# Function to fit ellipses and calculate their circumference
def fit_ellipses(binary_mask):
    binary_mask = binary_mask.copy()
    binary_mask = np.where(binary_mask != 0, 255, 0).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        return None

    # Select the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    if len(largest_contour) > 5:
        ellipse = cv2.fitEllipse(largest_contour)
        if not any(math.isnan(param) for param in ellipse[1]):
            a = ellipse[1][0] / 2
            b = ellipse[1][1] / 2
            circumference = ellipse_circumference(a, b)
            return circumference
    return None

# Function to load and preprocess MHA images
def load_mha_image(image_path, target_size):
    image = sitk.ReadImage(image_path)
    image_array = sitk.GetArrayFromImage(image)
    
    # Normalize and resize
    image_array = preprocess_image(image_array, (target_size, target_size))
    return image_array

# Function to normalize and resize images and masks
def preprocess_image(image_array, target_shape=(128, 128)):
    image_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array)) * 255.0
    return np.expand_dims(resize(image_array, target_shape, mode='constant', preserve_range=True), axis=-1)

# Function to load patient data
def load_patient_data(base_dir):
    print("Starting data loading...")
    image_paths = []
    mask_paths = []
    
    for patient_folder in os.listdir(base_dir):
        print(f"Processing folder: {patient_folder}")
        patient_path = os.path.join(base_dir, patient_folder)

        # Check if patient folder contains 'images' and 'masks' subfolders
        image_dir = os.path.join(patient_path, 'images')
        mask_dir = os.path.join(patient_path, 'masks')

        if os.path.exists(image_dir) and os.path.exists(mask_dir):
            for image_file in os.listdir(image_dir):
                # Check if corresponding mask file exists
                image_path = os.path.join(image_dir, image_file)
                mask_path = os.path.join(mask_dir, image_file)

                if os.path.exists(mask_path):
                    # Check if image and mask are not NaN
                    image_array = load_mha_image(image_path, SIZE)
                    mask_array = load_mha_image(mask_path, SIZE)

                    if np.isnan(image_array).any() or np.isnan(mask_array).any():
                        print(f"Image or mask has NaN values and will be removed: {image_file}")
                    else:
                        image_paths.append(image_path)
                        mask_paths.append(mask_path)
                else:
                    print(f"Mask for {image_file} in {patient_folder} not found!")
        else:
            print(f"'images' or 'masks' folders not found in {patient_folder}!")
    
    print("Data loading completed.")
    return image_paths, mask_paths

# Function to load a set of images or masks
def load_images(image_paths, SIZE):
    images = np.zeros((len(image_paths), SIZE, SIZE, 1))  # Single-channel images

    for i, image_path in enumerate(image_paths):
        images[i] = load_mha_image(image_path, SIZE)
    
    return images

# Function to calculate ellipse circumferences from masks
def calculate_ellipse_circumferences(mask_paths, masks):
    circumferences = []
    for i, mask in enumerate(masks):
        circumference = fit_ellipses(mask[:, :, 0])  # Remove extra dimension
        if circumference:
            circumferences.append((os.path.basename(mask_paths[i]), circumference))
        else:
            circumferences.append((os.path.basename(mask_paths[i]), None))
    return circumferences

# Main directory path and loading data
base_dir = r"H:\drproject\New folder"
SIZE = 128
image_paths, mask_paths = load_patient_data(base_dir)

# Load and preprocess masks
masks = load_images(mask_paths, SIZE)

# Calculate ellipse circumferences for masks
circumferences = calculate_ellipse_circumferences(mask_paths, masks)

# Save results to an Excel file
df = pd.DataFrame(circumferences, columns=['Filename', 'Circumference'])
df.to_excel("ellipse_circumferences.xlsx", index=False)

print("Excel file saved successfully!")

Starting data loading...
Processing folder: 2e99ee0e-506a-443b-8c16-e8a0619da2f7
Processing folder: bfd442e7-9299-4890-ad3d-c9d5b6534cbc
Processing folder: 0199616b-bdeb-4119-97a3-a5a3571bd641
Processing folder: dcdcb9a4-fec7-45a2-b2b7-a282f963c551
Processing folder: 484c03b0-441d-4966-b10e-06b31d84e55e
Processing folder: b9ef9a65-ab93-4714-a9e0-6df70ac8c285
Processing folder: d15a282b-25ca-40ab-915c-215335a0d3cf
Processing folder: e3670fa9-574c-4fbc-89af-f69aad63696d
Processing folder: 6fcb175f-52e4-4f3e-9956-f693b28f56e6
Processing folder: 7f689365-1782-4e41-b357-951d68cd3eee
Processing folder: 67d1dc32-1791-4595-937a-c51d4372c15c
Processing folder: 47540baa-7ab8-425a-9e77-8005cb1c02f4
Processing folder: 8ce68989-8856-4c3f-8988-4c9680b9a246
Processing folder: 229525cd-637a-4560-ad44-45b6f04a5d1f
Processing folder: f1fcabfc-f998-44c7-8420-c7a5ae5aaab7
Processing folder: 1b250c39-7217-4921-838b-ed2681ebfef2
Processing folder: 30555893-2ac6-45c9-b404-8ca2d9f90b05
Processing folder: ea72

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import cv2
import math
import SimpleITK as sitk
from skimage.transform import resize

# Function to calculate the circumference of an ellipse
def ellipse_circumference(a, b):
    return np.pi * ((3 * (a + b) - np.sqrt((3 * a + b) * (a + 3 * b))) * 0.28)

# Function to fit ellipses and calculate their circumference
def fit_ellipses(binary_mask):
    binary_mask = binary_mask.copy()
    binary_mask = np.where(binary_mask != 0, 255, 0).astype(np.uint8)

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        return None

    largest_contour = max(contours, key=cv2.contourArea)
    if len(largest_contour) > 5:
        ellipse = cv2.fitEllipse(largest_contour)
        if not any(math.isnan(param) for param in ellipse[1]):
            a = ellipse[1][0] / 2
            b = ellipse[1][1] / 2
            circumference = ellipse_circumference(a, b)
            return circumference
    return None

# Function to load and preprocess MHA images
def load_mha_image(image_path, target_size):
    image = sitk.ReadImage(image_path)
    image_array = sitk.GetArrayFromImage(image)
    
    # Normalize and resize
    image_array = preprocess_image(image_array, (target_size, target_size))
    return image_array

# Function to normalize and resize images
def preprocess_image(image_array, target_shape=(128, 128)):
    image_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array)) * 255.0
    return np.expand_dims(resize(image_array, target_shape, mode='constant', preserve_range=True), axis=-1)

# Function to load the trained model
def load_trained_model(model_path):
    return load_model(model_path)

# Function to predict masks
def predict_masks(model, images):
    predictions = model.predict(images)
    predicted_masks = (predictions > 0.5).astype(np.uint8)  # Convert predictions to binary
    return predicted_masks

# Function to calculate ellipse circumferences from masks
def calculate_ellipse_circumferences(mask_paths, predicted_masks):
    circumferences = []
    for i, mask in enumerate(predicted_masks):
        circumference = fit_ellipses(mask[:, :, 0])  # Remove extra dimension
        if circumference:
            circumferences.append((os.path.basename(mask_paths[i]), circumference))
        else:
            circumferences.append((os.path.basename(mask_paths[i]), None))
    return circumferences

# Define dice metric
def dice_metric(y_true, y_pred):
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)

# Define IoU metric
def iou_metric(y_true, y_pred):
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + 1.) / (union + 1.)

# Define F1 score
def f1_score(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))
    
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

# Define IoU loss
def iou_loss(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return 1 - iou

# Function to load the model with custom objects
def load_trained_model(model_path):
    return load_model(model_path, custom_objects={
        'dice_metric': dice_metric,
        'iou_metric': iou_metric,
        'f1_score': f1_score,
        'iou_loss': iou_loss
    })

# Main directory path and loading model
base_dir = r"H:\drproject\New folder"
model_path = r"H:\drproject\best_model_iou.h5"
SIZE = 128

# Load images
image_paths, mask_paths = load_patient_data(base_dir)  # Assuming this function is defined elsewhere
images = load_images(image_paths, SIZE)

# Load the trained model
model = load_trained_model(model_path)

# Predict masks with the model
predicted_masks = predict_masks(model, images)

# Calculate ellipse circumferences for predicted masks
circumferences = calculate_ellipse_circumferences(image_paths, predicted_masks)

# Save results to an Excel file
df = pd.DataFrame(circumferences, columns=['Filename', 'Circumference'])
df.to_excel("predicted_ellipse_circumferences.xlsx", index=False)

print("Excel file saved successfully!")

Starting data loading...
Processing folder: 2e99ee0e-506a-443b-8c16-e8a0619da2f7
Processing folder: bfd442e7-9299-4890-ad3d-c9d5b6534cbc
Processing folder: 0199616b-bdeb-4119-97a3-a5a3571bd641
Processing folder: dcdcb9a4-fec7-45a2-b2b7-a282f963c551
Processing folder: 484c03b0-441d-4966-b10e-06b31d84e55e
Processing folder: b9ef9a65-ab93-4714-a9e0-6df70ac8c285
Processing folder: d15a282b-25ca-40ab-915c-215335a0d3cf
Processing folder: e3670fa9-574c-4fbc-89af-f69aad63696d
Processing folder: 6fcb175f-52e4-4f3e-9956-f693b28f56e6
Processing folder: 7f689365-1782-4e41-b357-951d68cd3eee
Processing folder: 67d1dc32-1791-4595-937a-c51d4372c15c
Processing folder: 47540baa-7ab8-425a-9e77-8005cb1c02f4
Processing folder: 8ce68989-8856-4c3f-8988-4c9680b9a246
Processing folder: 229525cd-637a-4560-ad44-45b6f04a5d1f
Processing folder: f1fcabfc-f998-44c7-8420-c7a5ae5aaab7
Processing folder: 1b250c39-7217-4921-838b-ed2681ebfef2
Processing folder: 30555893-2ac6-45c9-b404-8ca2d9f90b05
Processing folder: ea72

In [3]:
import pandas as pd

# Load circumference data from Excel files
true_circumference_df = pd.read_excel(r"H:\drproject\ellipse_circumferences.xlsx")
pred_circumference_df = pd.read_excel(r"H:\drproject\predicted_ellipse_circumferences.xlsx")

# Reset index to ensure comparison is based on row order, not filename
true_circumference_df = true_circumference_df.reset_index(drop=True)
pred_circumference_df = pred_circumference_df.reset_index(drop=True)

# Ensure both dataframes have the same number of rows
min_length = min(len(true_circumference_df), len(pred_circumference_df))
true_circumference_df = true_circumference_df.iloc[:min_length]
pred_circumference_df = pred_circumference_df.iloc[:min_length]

# Merge the dataframes based on index (row order)
combined_df = pd.DataFrame({
    'Circumference_true': true_circumference_df['Circumference'],
    'Circumference_pred': pred_circumference_df['Circumference']
})

# Drop any rows with missing values (if any)
combined_df = combined_df.dropna()

# Calculate absolute and relative errors
combined_df['absolute_error'] = (combined_df['Circumference_true'] - combined_df['Circumference_pred']).abs()
combined_df['relative_error'] = combined_df['absolute_error'] / combined_df['Circumference_true']

# Calculate individual accuracies (1 - relative_error) for each row
combined_df['individual_accuracy'] = 1 - combined_df['relative_error']

# Print the number of data points
print(f"Number of data points: {len(combined_df)}")

# Calculate and print the final overall accuracy
overall_accuracy = combined_df['individual_accuracy'].mean()
print(f"Final overall accuracy: {overall_accuracy:.2%}")

# Save results to an Excel file
combined_df.to_excel('circumference_individual_accuracies.xlsx', index=False)

print("Results with individual accuracies saved to 'circumference_individual1_accuracies.xlsx'")


Number of data points: 2141
Final overall accuracy: 96.14%
Results with individual accuracies saved to 'circumference_individual1_accuracies.xlsx'
